# Functions

This page describes the different types of window functions, discusses the specifics of each group, and illustrates the key features in practice.

## Value

Value window functions (*рус. функции смещения*) allow each row in a group to access a value from another row within the same group. The following table shows the basic window functions.

| Function            | Description                                             |
| ------------------- | ------------------------------------------------------- |
| `LAG(column, n)`    | Returns the value from `n` rows before the current row. |
| `LEAD(column, n)`   | Returns the value from `n` rows after the current row.  |
| `FIRST_VALUE()`     | Returns the first value in the window frame.            |
| `LAST_VALUE()`      | Returns the last value in the window frame.             |
| `NTH_VALUE(col, n)` | Returns the `n`-th value in the window frame.           |

For more details on the value functions in SQL, check out the [value](functions/value.ipynb) page.

---

The following cell shows how value window functions works in the postgreSQL.

In [1]:
--postgreSQL
SELECT
    *,
    LAG("value", 1) OVER (PARTITION BY "group" ORDER BY "order"),
    LEAD("value", 1) OVER (PARTITION BY "group" ORDER BY "order"),
    FIRST_VALUE("value") OVER (PARTITION BY "group" ORDER BY "order"),
    LAST_VALUE("value") OVER (PARTITION BY "group" ORDER BY "order"),
    NTH_VALUE("value", 2) OVER (PARTITION BY "group" ORDER BY "order")
FROM (
    VALUES
    (3, 'C', 1),
    (3, 'B', 2),
    (1, 'B', 3),
    (4, 'C', 4),
    (3, 'A', 5),
    (5, 'C', 6),
    (6, 'B', 7),
    (7, 'A', 8)
) AS temp("order", "group", "value");

SELECT 8


order,group,value,lag,lead,first_value,last_value,nth_value
3,A,5,,8,5,5,
7,A,8,5,,5,8,8
1,B,3,,2,3,3,
3,B,2,3,7,3,2,2
6,B,7,2,,3,7,2
3,C,1,,4,1,1,
4,C,4,1,6,1,4,4
5,C,6,4,,1,6,4
